In [22]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords 

In [43]:
df = pd.read_csv('dataset/cik_list.csv')

In [44]:
df.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
0,3662.0,SUNBEAM CORP/FL/,199803.0,3/6/1998,10-K405,edgar/data/3662/0000950170-98-000413.txt
1,3662.0,SUNBEAM CORP/FL/,199805.0,5/15/1998,10-Q,edgar/data/3662/0000950170-98-001001.txt
2,3662.0,SUNBEAM CORP/FL/,199808.0,8/13/1998,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt
3,3662.0,SUNBEAM CORP/FL/,199811.0,11/12/1998,10-K/A,edgar/data/3662/0000950170-98-002145.txt
4,3662.0,SUNBEAM CORP/FL/,199811.0,11/16/1998,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt


There are some blank rows. Need to delete them.

In [9]:
df = df.dropna(axis='index', how='all')

We have 152 records and 6 columns initially.

In [45]:
df.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
0,3662.0,SUNBEAM CORP/FL/,199803.0,3/6/1998,10-K405,edgar/data/3662/0000950170-98-000413.txt
1,3662.0,SUNBEAM CORP/FL/,199805.0,5/15/1998,10-Q,edgar/data/3662/0000950170-98-001001.txt
2,3662.0,SUNBEAM CORP/FL/,199808.0,8/13/1998,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt
3,3662.0,SUNBEAM CORP/FL/,199811.0,11/12/1998,10-K/A,edgar/data/3662/0000950170-98-002145.txt
4,3662.0,SUNBEAM CORP/FL/,199811.0,11/16/1998,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt


A utility method to get a URL

In [4]:
def get_page(url):
    page = requests.get(url)
    return page

In [ ]:
A utility method to get a soup object using

In [2]:
def get_soup(page):
    return BeautifulSoup(page.content, 'html.parser')

URL to the report repository

In [12]:
url = 'https://www.sec.gov/Archives/'

We concatenate this common urlwith individual resources as follows

In [14]:
url + df.iloc[0][-1]

'https://www.sec.gov/Archives/edgar/data/3662/0000950170-98-000413.txt'

Now, we will traverse each row of the DataFrame, fetch the resource at the concatenated url

We create a new DataFrame for variables only. We will later merge it with the original DataFrame to append records for individual rows.

In [34]:
df_variables = pd.DataFrame(columns=['mda_positive_score', 'mda_negative_score', 'mda_polarity_score', 'mda_average_sentence_length'
                                    'mda_percentage_of_complex_words', 'mda_fog_index', 'mda_complex_word_count', 'mda_word_count',
                                    'mda_uncertainty_score', 'mda_constraining_score', 'mda_positive_word_proportion', 'mda_negative_word_proportion',
                                    'mda_uncertainty_word_proportion', 'mda_constraining_word_proportion', 'qqdmr_positive_score', 'qqdmr_negative_score',
                                    'qqdmr_polarity_score', 'qqdmr_average_sentence_length', 'qqdmr_percentage_of_complex_words', 'qqdmr_fog_index',
                                    'qqdmr_complex_word_count', 'qqdmr_word_count', 'qqdmr_uncertainty_score', 'qqdmr_constraining_score', 'qqdmr_positive_word_proportion',
                                    'qqdmr_negative_word_proportion', 'qqdmr_uncertainty_word_proportion', 'qqdmr_constraining_word_proportion', 'rf_positive_score',
                                    'rf_negative_score', 'rf_polarity_score', 'rf_average_sentence_length', 'rf_percentage_of_complex_words', 'rf_fog_index',
                                    'rf_complex_word_count', 'rf_word_count', 'rf_uncertainty_score', 'rf_constraining_score', 'rf_positive_word_proportion',
                                    'rf_negative_word_proportion', 'rf_uncertainty_word_proportion', 'rf_constraining_word_proportion', 'constraining_words_whole_report'
                                    ])

In [35]:
df_variables

,mda_positive_score,mda_negative_score,mda_polarity_score,mda_average_sentence_lengthmda_percentage_of_complex_words,mda_fog_index,mda_complex_word_count,mda_word_count,mda_uncertainty_score,mda_constraining_score,mda_positive_word_proportion,...,rf_fog_index,rf_complex_word_count,rf_word_count,rf_uncertainty_score,rf_constraining_score,rf_positive_word_proportion,rf_negative_word_proportion,rf_uncertainty_word_proportion,rf_constraining_word_proportion,constraining_words_whole_report


Reading Loughran McDonald Dictionary

In [29]:
loughran_mcdonald = pd.read_csv('dataset/LoughranMcDonald_MasterDictionary_2018.csv')

In [41]:
loughran_mcdonald.head()

,Word,Sequence Number,Word Count,Word Proportion,Average Proportion,Std Dev,Doc Count,Negative,Positive,Uncertainty,Litigious,Constraining,Superfluous,Interesting,Modal,Irr_Verb,Harvard_IV,Syllables,Source
0,AARDVARK,1,277,1.480000e-08,1.240000e-08,3.560000e-06,84,0,0,0,0,0,0,0,0,0,0,2,12of12inf
1,AARDVARKS,2,3,1.600000e-10,9.730000e-12,9.860000e-09,1,0,0,0,0,0,0,0,0,0,0,2,12of12inf
2,ABACI,3,8,4.280000e-10,1.390000e-10,6.230000e-08,7,0,0,0,0,0,0,0,0,0,0,3,12of12inf
3,ABACK,4,12,6.410000e-10,3.160000e-10,9.380000e-08,12,0,0,0,0,0,0,0,0,0,0,2,12of12inf
4,ABACUS,5,7250,3.870000e-07,3.680000e-07,3.370000e-05,914,0,0,0,0,0,0,0,0,0,0,3,12of12inf


Reading Constraining Words Dictionary

In [25]:
constraining_dictionary = pd.read_csv('dataset/constraining_dictionary.csv')

Reading Uncertainty Words Dictionary

In [27]:
uncertainty_dictionary = pd.read_csv('dataset/uncertainty_dictionary.csv')

We create three utility methods for extracting three sections in a particular document. The sections are:
1. "Management's Discussion and Analysis": mda()
2. "Quantitative and Qualitative Disclosures about Market Risk": qqdmr()
3. "Risk Factors": rf()

In [ ]:
def GetVariables():
    
    def __init__(self, page, mda, qqdmr, rf):
        self.page = page
        self.mda = mda
        self.qqdmr = qqdmr
        self.rf = rf
        
    def remove_punctuation(self):
        return ''.join([char for char in self.text if char not in punctuation])
        
    def tokenize(self):
        return word_tokenize(self.text)
    
    def remove_stop_words(self):
        after_stop_words_removal = [word for word in self.tokenize(word) if word not in stopwords.words('english')]
        
    def preprocess(self):
        self.text = self.remove_punctuation()
        self.text = self.remove_stop_words()
        return self.text
    
    def mda(self):
        desired_string = re.split(".\s{0,2}MANAGEMENT'S DISCUSSION AND ANALYSIS", self.page, re.IGNORECASE)
        if desired_string[0] == self.page:
            return None
        desired_string = re.split("ITEM \d", desired_string[-1], re.IGNORECASE)[0]
        desired_string = re.split("OF OPERATIONS?", desired_string, re.IGNORECASE)
        self.mda = desired_string[-1]
    
    def qqdmr(self):
        desired_string = re.split(".\s{0,2}QUANTITATIVE AND QUALITATIVE", self.page, re.IGNORECASE)
        if desired_string[0] == page:
            return None
        desired_string = re.split("ITEM", desired_string[-1], re.IGNORECASE)[0]
        desired_string = re.split("ABOUT MARKET RISK", desired_string, re.IGNORECASE)
        self.qqdmr desired_string[-1]
        
    def rf(self):
        desired_string = re.split(".\s{0,2} RISK FACTORS", self.page, re.IGNORECASE)
        if desired_string[0] == page:
            return None
        print(desired_string)
        desired_string = re.split("ITEM \d", desired_string[-1], re.IGNORECASE)[0]
        self.rf = desired_string
        
    def word_count(self, section):
        return section.split(' ')
        
    def sentence_count(self, section):
        return section.split('. ')
    
    def complex_word_count(self, section):
        sentence_list = self.num_of_sentences(section)
        complex_word_count = 0
        for word in sentence_list:
            if word[-2:] == 'es':
                complex_word_count -= 1 
            if word[-2:] == 'ed':
                complex_word_count -= 1
            for char in word:
                if char in 'aeiou':
                    complex_word_count += 1
        return complex_word_count
    
    def positive_score(self, section):
        positive_score = 0
        word_list = self.word_count(section)
        for word in word_list:
            if not loughran_mcdonald[loughran_mcdonald['Word'] == word]['Positive'].empty:
                positive_score += 1
        return positive_score
        
    def negative_score(self, section):
        negative_score = 0
        word_list = section.split()
        for word in word_list:
            if not loughran_mcdonald[loughran_mcdonald['Word'] == word]['Negative'].empty:
                negative_score += 1
        return negative_score
        
    def polarity_score(self, positive_score, negative_score):
        return (positive_score – negative_score) / ((positive_score + negative_score) + 0.000001)
        
    def average_sentence_length(self, section):
        return self.word_count(section) / self.sentence_count(section)
        
    def percentage_of_complex_words(self, section):
        self.complex_word_count(section) / self.word_count(section)
        
    def fog_index(self, section):
        return 0.4 * (self.average_sentence_length(section) + self.percentage_of_complex_words(section))
        
    def uncertainty_score(self, section):
        uncertainty_score = 0
        word_list = self.word_count(section)
        for word in word_list:
            if word in uncertainty_dictionary.values:
                uncertainty_score += 1
        return uncertainty_score
        
    def constraining_score(self, section):
        constraining_score = 0
        word_list = self.word_count(section)
        for word in word_list:
            if word in constraining_dictionary.values:
                constraining_score += 1
        return constraining_score
        
    def positive_word_proportion(self, positive_score, word_count):
        return positive_score / word_count
        
    def negative_word_proportion(self, negative_score, word_count):
        return positive_score / word_count        
        
    def uncertainty_word_proportion(self, uncertainty_score, word_count):
        return uncertainty_score / word_count
        
    def constraining_word_proportion(self, constraining_score, word_count):
        return constraining_score / word_count    
         
    def constraining_words_whole_report(self, page):
        constraining_words_whole_report = 0
        word_list = self.word_count(page)
        for word in word_list:
            if word in constraining_dictionary.values:
                constraining_words_whole_report += 1
        return constraining_words_whole_report 
    
    def get_variables():
        
        # Invoking methods for MDA and storing values in variables serving as DataFrame Columns.
        mda_positive_score = self.positive_score(self.page, 'mda')
        self.list.append(mda_positive_score)
        self.list.append(self.negative_score(self.page, 'mda'))
        self.list.append(self.polarity_score(self.page, 'mda'))
        self.list.append(self.average_sentence_length(self.page, 'mda'))
        self.list.append(self.percentage_of_complex_words(self.page, 'mda'))
        self.list.append(self.fog_index(self.page, 'mda'))
        self.list.append(self.complex_word_count(self.page, 'mda'))
        mda_word_count = self.word_count(self.page, 'mda')
        self.list.append(mda_word_count)
        self.list.append(self.uncertainty_score(self.page, 'mda'))
        self.list.append(self.constraining_score(self.page, 'mda'))   
        self.list.append(self.positive_word_proportion(self.page, mda_positive_score, mda_word_count))
        mda_negative_score = self.negative_score(self.page, 'mda')
        self.list.append(self.negative_word_proportion(self.page, mda_negative_score, mda_word_count))
        mda_uncertainty_score = self.uncertainty_score(self.page, 'mda')
        self.list.append(self.uncertainty_word_proportion(self.page, mda_uncertainty_score, mda_word_count))
        mda_constraining_score = self.constraining_score(self.page, 'mda')
        self.list.append(self.constraining_word_proportion(self.page, mda_constraining_score, mda_word_count))
        
        # Invoking methods for QQDMR and storing values in variables serving as DataFrame Columns.
        qqdmr_positive_score = self.positive_score(self.page, 'qqdmr')
        self.list.append(qqdmr_positive_score)
        self.list.append(self.negative_score(self.page, 'qqdmr'))
        self.list.append(self.polarity_score(self.page, 'qqdmr'))
        self.list.append(self.average_sentence_length(self.page, 'qqdmr'))
        self.list.append(self.percentage_of_complex_words(self.page, 'qqdmr'))
        self.list.append(self.fog_index(self.page, 'qqdmr'))
        self.list.append(self.complex_word_count(self.page, 'qqdmr'))
        qqdmr_word_count = self.word_count(self.page, 'qqdmr')
        self.list.append(qqdmr_word_count)
        self.list.append(self.uncertainty_score(self.page, 'qqdmr'))
        self.list.append(self.constraining_score(self.page, 'qqdmr'))   
        self.list.append(self.positive_word_proportion(self.page, qqdmr_positive_score, qqdmr_word_count))
        qqdmr_negative_score = self.negative_score(self.page, 'qqdmr')
        self.list.append(self.negative_word_proportion(self.page, qqdmr_negative_score, qqdmr_word_count))
        qqdmr_uncertainty_score = self.uncertainty_score(self.page, 'qqdmr')
        self.list.append(self.uncertainty_word_proportion(self.page, qqdmr_uncertainty_score, qqdmr_word_count))
        qqdmr_constraining_score = self.constraining_score(self.page, 'qqdmr')
        self.list.append(self.constraining_word_proportion(self.page, qqdmr_constraining_score, qqdmr_word_count))
        
        # Invoking methods for RF and storing values in variables serving as DataFrame Columns.
        mda_positive_score = self.positive_score(self.page, 'rf')
        self.list.append(rf_positive_score)
        self.list.append(self.negative_score(self.page, 'rf'))
        self.list.append(self.polarity_score(self.page, 'rf'))
        self.list.append(self.average_sentence_length(self.page, 'rf'))
        self.list.append(self.percentage_of_complex_words(self.page, 'rf'))
        self.list.append(self.fog_index(self.page, 'rf'))
        self.list.append(self.complex_word_count(self.page, 'rf'))
        rf_word_count = self.word_count(self.page, 'rf')
        self.list.append(rf_word_count)
        self.list.append(self.uncertainty_score(self.page, 'rf'))
        self.list.append(self.constraining_score(self.page, 'rf'))   
        self.list.append(self.positive_word_proportion(self.page, rf_positive_score, rf_word_count))
        rf_negative_score = self.negative_score(self.page, 'rf')
        self.list.append(self.negative_word_proportion(self.page, rf_negative_score, rf_word_count))
        rf_uncertainty_score = self.uncertainty_score(self.page, 'rf')
        self.list.append(self.uncertainty_word_proportion(self.page, rf_uncertainty_score, rf_word_count))
        rf_constraining_score = self.constraining_score(self.page, 'rf')
        self.list.append(self.constraining_word_proportion(self.page, rf_constraining_score, rf_word_count))
        self.list.append(self.constraining_words_whole_report(self.page))
        
        # Returning a list of variables serving as DataFrame Columns.
        return self.list

Now we visit each url, store its content in an HTML file 'document.html', use methods in the class 'GetVariables' to
extract out sections and for each section, get the metrics, save them to a list of variables. Finally we append that list
to a new DataFrame 'df_variables' and eventually merge it to original DataFrame 'df' which only consisted of Company Details originally.

In [ ]:
for row in df.iterrows():
    # Fetch a URL
    page = get_page('https://www.sec.gov/Archives/' + df.loc[i][-1])
    # The fetched URL is unparsed HTML. Need to store it in an HTML file, then visit the new URL, and apply soup to it.
    write_document = open('document.html', 'wb').write(page.content)
    page = open('document.html', 'r').read()
    df_variables.loc[len(df_variables)] = GetVariables(page).get_variables()
df = pd.concat([df, df_variables], axis=1)